# Modèle Harder LASSO linéaire (fonction de coût square-root $\ell_2$ et pénalité Hard)

In [ ]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent.parent.parent)+"\AYMEN\EXPLORATION\CAS LINEAIRE\OUTILS")